In [1]:
from data import UnraveledDataset
from data import load_data
from models import MLP
from train import *
import torch
from torchvision import datasets
import utils_graphs as ug
from utils_tda import *

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

'cuda'

In [4]:
dataclass = UnraveledDataset
dataset = datasets.MNIST
batch_size = 1500
dataloader_train, dataloader_val = load_data(dataclass, dataset, download = False)

In [5]:
kwargs = dict(
    n_features=28 * 28,
    hidden_layer_sizes=(100, 100),
    n_targets=10
   # root = 'networks/mlp/MNISTunder'
)
loss_inst = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

In [6]:
model = MLP(**kwargs)
model.to(device)

MLP(
  (module_list): ModuleList(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [7]:
train(model, dataloader_train, dataloader_val, loss = loss_inst, optimizer = optimizer, device = device)

Train accuracy this epoch = 0.7230499982833862
Validation accuracy this epoch = 0.8482999801635742
Validation loss this epoch = 0.5405658483505249
Train accuracy this epoch = 0.8684999942779541
Validation accuracy this epoch = 0.8916999697685242
Validation loss this epoch = 0.38105377554893494
Train accuracy this epoch = 0.8978833556175232
Validation accuracy this epoch = 0.9088999629020691
Validation loss this epoch = 0.3220166563987732
Train accuracy this epoch = 0.9105166792869568
Validation accuracy this epoch = 0.9187999963760376
Validation loss this epoch = 0.2806236147880554
Train accuracy this epoch = 0.9207833409309387
Validation accuracy this epoch = 0.9250999689102173
Validation loss this epoch = 0.249092698097229
Train accuracy this epoch = 0.9295666813850403
Validation accuracy this epoch = 0.9348999857902527
Validation loss this epoch = 0.22623878717422485
Train accuracy this epoch = 0.9345499873161316
Validation accuracy this epoch = 0.9380999803543091
Validation loss th

In [8]:
model.to('cpu')

MLP(
  (module_list): ModuleList(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [9]:
batch = next(iter(dataloader_val))

In [10]:
n_obs_per_class = 100

labels = np.unique(batch[1])
# Take n_obs_per_class observation from each class in x_train:
xs = [batch[0][np.where(batch[1] == label)][:n_obs_per_class] for label in labels]

# Now we get the corresponding barycenters.


In [11]:
%%time
#layers_id = [2]
all_barycenters = [barycenters_of_set_from_deep_model(model, x, layers_id= None) for x in xs]

CPU times: total: 3min 42s
Wall time: 3min 26s


In [12]:
n_new_obs = 10
val_batch = next(iter(dataloader_val))
new_x = val_batch[0][:n_new_obs]
new_y = val_batch[1][:n_new_obs]

In [24]:
res = topological_uncertainty(model, new_x, all_barycenters, layers_id= None)

In [25]:
res

array([0.00223714, 0.00994979, 0.00397282, 0.00345757, 0.00499296,
       0.00964257, 0.01026556, 0.00519569, 0.00985126, 0.00143115])

In [18]:
for i in range(100):
    new_new_x = new_x + torch.tensor(np.array([np.random.normal(scale = 00.1*i, size = 784) for j in range(10)])).float()
    res = topological_uncertainty(model, new_new_x, all_barycenters, layers_id= None)
    print(res)

[0.00223714 0.00994979 0.00397282 0.00345757 0.00499296 0.00964257
 0.01026556 0.00519569 0.00985126 0.00143115]
[0.00336868 0.00860958 0.00360528 0.00297999 0.00514581 0.00847632
 0.00980254 0.00542847 0.00969801 0.00224953]
[0.00449074 0.00775871 0.00365734 0.00549667 0.0058796  0.0103994
 0.0073833  0.00446562 0.01071721 0.00434594]
[0.0034808  0.00595979 0.00806192 0.01025005 0.00961442 0.00933778
 0.00764831 0.00329202 0.00454878 0.00300005]
[0.00317254 0.00811683 0.01227569 0.00713474 0.00586037 0.0163186
 0.00713049 0.01277896 0.00414804 0.00574983]
[0.02185445 0.00763687 0.00641441 0.00843743 0.00745916 0.01346601
 0.01055409 0.00725851 0.00599443 0.00793691]
[0.01543886 0.01805856 0.01068794 0.00570175 0.01204438 0.01282228
 0.01077144 0.00513575 0.00660677 0.0036803 ]
[0.01132758 0.01492209 0.00472648 0.01137726 0.01333794 0.01432869
 0.00759445 0.0108066  0.0164177  0.01040154]
[0.0119536  0.01265223 0.04014807 0.00572224 0.02500718 0.01088222
 0.00801401 0.01080377 0.013177

In [19]:
fdataclass = UnraveledDataset
fdataset = datasets.FashionMNIST
fbatch_size = 100
_, fmnist_x = load_data(fdataclass, fdataset, download = False, batch_size = fbatch_size)

In [20]:
fbatch_x = next(iter(fmnist_x))[0][:10]

In [21]:
%%time
res2 = topological_uncertainty(model, fbatch_x, all_barycenters, layers_id= None)

CPU times: total: 3.19 s
Wall time: 2.07 s


In [22]:
res2

array([0.05192733, 0.05926341, 0.035146  , 0.01556456, 0.01711458,
       0.04421658, 0.01001919, 0.07292169, 0.01203476, 0.02975044])